# Numerical Analysis Final Project: Lasso vs Ridge Regression

In [2]:
import Pkg
Pkg.add("Pandas")

    Updating registry at `C:\Users\Kingston\.julia\registries\General.toml`
   Resolving package versions...
   Installed TableTraitsUtils ─ v1.0.2
   Installed DataValues ─────── v0.4.13
   Installed Lazy ───────────── v0.15.1
   Installed Pandas ─────────── v1.5.3
   Installed PyCall ─────────── v1.93.1
    Updating `C:\Users\Kingston\.julia\environments\v1.7\Project.toml`
  [eadc2687] + Pandas v1.5.3
    Updating `C:\Users\Kingston\.julia\environments\v1.7\Manifest.toml`
  [e7dc6d0d] + DataValues v0.4.13
  [50d2b5c4] + Lazy v0.15.1
  [eadc2687] + Pandas v1.5.3
  [438e738f] + PyCall v1.93.1
  [382cd787] + TableTraitsUtils v1.0.2
    Building PyCall → `C:\Users\Kingston\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\1fc929f47d7c151c839c5fc1375929766fb8edcc\build.log`
Precompiling project...
  ✓ DataValues
  ✓ TableTraitsUtils
  ✓ Lazy
  ✓ PyCall
  ✓ Pandas
  5 dependencies successfully precompiled in 9 seconds (156 already precompiled, 3 skipped during auto due to previous 

In [3]:
using Pkg
using Pandas
using LinearAlgebra

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Kingston\.julia\conda\3

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandas-1.4.2               |   py39h2e25243_1        11.0 MB  conda-forge
    python-dateutil-2.8.2      |     pyhd8ed1ab_0         240 KB  conda-forge
    pytz-2022.1                |     pyhd8ed1ab_0         242 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        11.5 MB

The following NEW packages will be INSTALLED:

  pandas             conda-forge/win-64::pandas-1.4.2-py39h2e25243_1
  python-dateutil    conda-forge/noarch::python-dateutil-2.8.2-pyhd8ed1ab_0
  pytz               conda-forge/noarch::pytz-2022.1-pyhd8ed1ab_0



pandas-1.4.2         | 11.0 MB   | ########## | 100% 
python-dateutil-2

┌ Info: Installing pandas via the Conda pandas package...
└ @ PyCall C:\Users\Kingston\.julia\packages\PyCall\7a7w0\src\PyCall.jl:711
┌ Info: Running `conda install -y pandas` in root environment
└ @ Conda C:\Users\Kingston\.julia\packages\Conda\1403Y\src\Conda.jl:129


## Import data

In [4]:
df = read_csv("median_housing_cost_data.tsv", sep="\t");

In [5]:
x = iloc(df)[0]

Unnamed: 0    20639.000000
MedInc            2.388600
HouseAge         16.000000
AveRooms          5.254717
AveBedrms         1.162264
Population     1387.000000
AveOccup          2.616981
Latitude         39.370000
Longitude      -121.240000
Name: 20639, dtype: float64

## Motivating factor

In [ ]:
# talk about median household costs and how maximizing posterior probabilities to estimate cost is goal

## Regression tasks derived from maximizing posterior probabilities

## Solutions to regression tasks

## Code solutions for regression tasks

In [49]:
function linear_regression_ridge(X,y,lambda)
    
    s = size(X)[1]
    Im =1 * Matrix(I, s, s)
    
    w = inv(transpose(X)*X + lambda*Im) * transpose(X) * y
    
    pred = X * w
    
    err = y - pred
    
    return w, pred, err    
    
end

linear_regression_ridge (generic function with 1 method)

In [ ]:
function linear_regression_lasso(X,y,lambda)
    
    
end

In [6]:
function linear_regression_test(X,w)
    pred = X * w
    return pred
end

linear_regression_test (generic function with 1 method)

## Analysis of algorithms: conditioning, complexities, and flop counts

In [ ]:
# Time Complexity Calculation

In [ ]:
# Complexity of Ridge Regression

In [ ]:
function linear_regression_ridge(X,y,lambda) # Size of X is n x m
    s  = size(X)[1]                          # O(n) or O(1)?
    Im = 1 * Matrix(I, s, s)                 # O(n^2)
    xT = transpose(X)                        # O(n^2), O(n) <https://ieeexplore.ieee.org/document/6131813>, or O(1) ??? 
    xT_lambda = (xT * X) + (lambda * Im)     # Matrix Multiplication = O(n*m*p) or O(n^3) (square)
    inverse = inv(xT_lambda)                 # Inverse: O(???)
    w = inverse * xT * y                     # Matrix Multiplication = O(n*m*p) or O(n^3) (x2) 
    pred = X * w                             # Matrix Multiplication = O(n^3)
    err = y - pred                           # Matrix Subtraction O(n^2)
    
    return w, pred, err                      # O(1)
end

# Total Time Complexity = O(n^3) (Most Likely)

In [ ]:
# Space Complexity Calculation

In [ ]:
# Flop Count Calculation

In [ ]:
# Conditioning of Algorithm

## Perform regression tasks on data

## Evaluation of algorithm performance

## Conclusion